# 1.문제 정의

In [ ]:
import numpy as np
import pandas as pd

# 데이터 불러오기
data = pd.read_csv('/content/drive/MyDrive/2023-2알고리즘/[Project] 주어진 행렬 공간 중에서 최대 volume 구하기/input.csv')
vectors = data.iloc[1:21].to_numpy()  # 첫 행은 인덱스, 나머지가 벡터 값

# 초기 설정
num_vectors = 20    # 행렬의 행 수
population_size = 100  # 인구 크기
vector_length = 10000  # 전체 벡터의 수 설정 (예시로 10000으로 가정)
num_generations = 100  # 세대 수
mutation_rate = 0.1   # 돌연변이율
print(vectors)

[[ 0.09130457  0.31296659 -0.3277619  ...  0.17386052 -0.49610621
   0.95447834]
 [-1.94118936  0.63436125  0.21332334 ...  2.63566329  0.98461692
  -2.05137763]
 [ 1.20599072  0.63773173  0.37202377 ...  0.1490941  -0.16698097
   0.70955588]
 ...
 [-1.51843521  0.6081658   0.01824384 ...  2.1008383  -0.13667624
  -0.88527608]
 [ 1.31471571 -1.06830117 -0.25633056 ... -1.24862182  0.09704474
   1.50861745]
 [ 1.23136462 -0.02520689  0.74379476 ...  0.345985   -2.36028857
   1.78184592]]


# 2. 초기 인구 생성


In [ ]:
def initialize_population(pop_size, num_vectors):
    # x 좌표를 0부터 19 사이에서 선택
    x_indices = np.random.randint(0, num_vectors-1, size=(pop_size, num_vectors))

    # y 좌표를 0부터 9999 사이에서 선택
    y_indices = np.random.randint(0, vector_length-1, size=(pop_size, num_vectors))

    # x와 y 좌표를 결합하여 2차원 인덱스 생성
    indices = np.stack((x_indices, y_indices), axis=-1)

    return indices


population = initialize_population(population_size, num_vectors)
print("population size: ", population.shape)
print(population)

population size:  (100, 20, 2)
[[[   7 1628]
  [   1 3510]
  [  16 8227]
  ...
  [   3  553]
  [  18 2946]
  [  14 1708]]

 [[   7 5255]
  [  10 9175]
  [  16 6648]
  ...
  [  15 8159]
  [  14 4677]
  [  15 2392]]

 [[  18 2017]
  [  14 3538]
  [   2   91]
  ...
  [  15 7136]
  [   8 9253]
  [  17 1139]]

 ...

 [[  17 3833]
  [   8 7499]
  [  15 3816]
  ...
  [  12 5391]
  [   4 4853]
  [  14 1072]]

 [[  14  223]
  [  12 3490]
  [   6 8707]
  ...
  [  10 3158]
  [   4 9690]
  [   8 4592]]

 [[   4 6813]
  [  13 1624]
  [  14 5871]
  ...
  [   8 9606]
  [   0  733]
  [  16 2150]]]


# 3. 적합도 함수 정의


In [ ]:
def fitness(matrix):
    return np.abs(np.linalg.det(matrix))

def evaluate_population(population, vectors):
    fitness_scores = []
    for indices in population:
        matrix = vectors[:, indices]
        fitness_scores.append(fitness(matrix))
    return np.array(fitness_scores)

fitness_scores = evaluate_population(population, vectors)


LinAlgError: ignored

# 4. 선택

In [ ]:
def select_parents(population, fitness_scores):
    probabilities = fitness_scores / fitness_scores.sum()
    selected_indices = np.random.choice(len(population), size=len(population), replace=True, p=probabilities)
    return population[selected_indices]

parents = select_parents(population, fitness_scores)


# 5. 교차

In [ ]:
def crossover(parents):
    offspring = []
    for i in range(0, len(parents), 2):
        parent1 = parents[i]
        parent2 = parents[i + 1]
        cut = np.random.randint(1, len(parent1)-1)
        child1 = np.concatenate([parent1[:cut], parent2[cut:]])
        child2 = np.concatenate([parent2[:cut], parent1[cut:]])
        offspring.extend([child1, child2])
    return np.array(offspring)

offspring = crossover(parents)


# 6. 돌연변이

In [ ]:
# 돌연변이 함수
def mutation(offspring_crossover, mutation_rate, num_vectors):
    for idx in range(offspring_crossover.shape[0]):
        if np.random.rand() < mutation_rate:
            random_index = np.random.randint(offspring_crossover.shape[1])
            random_value = np.random.randint(num_vectors)
            offspring_crossover[idx, random_index] = random_value
    return offspring_crossover
# 돌연변이 함수 호출
num_vectors = 10000  # 전체 벡터의 수 설정
mutated_offspring = mutation(offspring, mutation_rate, num_vectors)


# 7-8. 새로운 세대 생성 및 종료 조건 검사

In [ ]:
for generation in range(num_generations):
    fitness_scores = evaluate_population(population, vectors)
    parents = select_parents(population, fitness_scores)
    offspring = crossover(parents)  # 여기서 두 번째 인자 제거
    mutated_offspring = mutation(offspring, mutation_rate, num_vectors)
    population = mutated_offspring
    # 종료 조건 검사
    # ...


# 9. 결과 추출

In [ ]:
final_fitness = evaluate_population(population, vectors)
best_index = np.argmax(final_fitness)
best_solution = population[best_index]
best_volume = final_fitness[best_index]

print("최대 볼륨:", best_volume)
print("해당 벡터의 인덱스:", best_solution)
